In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import sparse
import time

from ripser import ripser
from persim import plot_diagrams
import spectrum_distance_sup_operator_l1 as sd
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import fowlkes_mallows_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import AffinityPropagation

In [2]:
def is_inf(x):
   return x[1]!=np.inf


In [3]:
def get_diagram_delete_inf(data):
    N = len(data)
    #print("data",data)
    #print("len(data)",N)
    #print(data)
    I = np.arange(N-1)
    J = np.arange(1, N)
    V = np.maximum(data[0:-1], data[1::])
    #print("V",V)
    I = np.concatenate((I, np.arange(N)))
    #print("I.shape",I.shape)
    J = np.concatenate((J, np.arange(N)))
    V = np.concatenate((V, data))
    #print("V",V)
    #print("V.shape",V.shape)
    D = sparse.coo_matrix((V, (I, J)), shape=(N, N)).tocsr()
    #print(D)
    dgm0 = ripser(D, maxdim=0, distance_matrix=True)['dgms'][0]
    dgm0 = dgm0.tolist()
    dgm0_delete_inf = list(filter(is_inf, dgm0))
    return dgm0_delete_inf

In [4]:

sample_rate_1, data_1 = wavfile.read('wav//220747__jmuehlhans__tuning-fork-440-hz-resonance-box.wav')
sample_rate_2, data_2 = wavfile.read('wav//334536__teddy_frost__piano-normal-d4.wav')
if data_1.ndim > 1:
    data_1 = data_1[:, 0]
if data_2.ndim > 1:
    data_2 = data_2[:, 0]
#print(data_1)
#print(data_2)
#piece_1_start = np.random.uniform(3,20)
#piece_2_start = np.random.uniform(0.1,1.3)


from tqdm import tqdm
mean_acl_num_cluster = 0
mean_ap_num_cluster = 0
mean_acl_score = 0
mean_ap_score = 0
mean_km_score = 0
for i in tqdm(range(100), desc='Spectrum'):
    tuning_class = []
    for i in range(100):
        #piece_1_start = np.random.uniform(4,10)
        piece_1_start = np.random.uniform(3,15)
        piece_1_length = np.random.uniform(0.02,0.05)
        x_1 = data_1[int(piece_1_start* sample_rate_1):int((piece_1_start+piece_1_length)* sample_rate_1)]
        get_dgm1_delete_inf = get_diagram_delete_inf(x_1)
        tuning_class.append(get_dgm1_delete_inf)
    #print(len(tuning_class))
    #print(tuning_class)
    piano_class = []
    for j in range(100):
        piece_2_start = np.random.uniform(0.2,1.0)
        #piece_2_start = np.random.uniform(0.1,1.3)
        piece_2_length = np.random.uniform(0.02,0.05)
        #print(piece_2_start)
        #print(int(piece_2_start* sample_rate_2))
        x_2 = data_2[int(piece_2_start* sample_rate_2):int((piece_2_start+piece_2_length) * sample_rate_2)]
        #print(x_2)
        get_dgm2_delete_inf = get_diagram_delete_inf(x_2)
        piano_class.append(get_dgm2_delete_inf)

    datas = []
    datas.extend(tuning_class)
    datas.extend(piano_class)
    labels = np.zeros(200)
    labels[100:] = 1

    """
    generate distance matrix
    """

    def get_sc_dismatrix(datas):
        distance_matrix = np.zeros((len(datas), len(datas)))
        #print("distance_matrix.shape", distance_matrix.shape)
        for i in range(len(datas)):
            for j in range(len(datas)):
                distance_matrix[i, j] = sd.compute_distance(datas[i], datas[j])
        return distance_matrix

    A = get_sc_dismatrix(datas)

    """
    Agglomerative Clustering
    """


    acl = AgglomerativeClustering(n_clusters=2, metric='precomputed', linkage='average')

    acl_labels = acl.fit_predict(A)
    mean_acl_num_cluster += len(set(acl_labels))



    acl_score = fowlkes_mallows_score(labels, acl_labels)
    print("acl_score",acl_score)
    mean_acl_score += acl_score

    """
    Affinity Propagation
    """


    ap = AffinityPropagation(affinity='precomputed')
    ap_clustering = ap.fit(A)
    ap_labels = ap_clustering.labels_

    mean_ap_num_cluster+=len(set(ap_labels))
    print("len(set(ap_labels))",len(set(ap_labels)))
   

    ap_score = fowlkes_mallows_score(labels, ap_labels)
    print("ap_score",ap_score)
    mean_ap_score += ap_score

    """
    k-med
    """
    kmedoids = KMedoids(n_clusters=2, random_state=0).fit(A)
    #print(kmedoids.labels_)
    km_score = fowlkes_mallows_score(labels, kmedoids.labels_)
    print("km_score",km_score)
    mean_km_score+=km_score
average_acl_cluster = mean_acl_num_cluster/100
average_acl_score = mean_acl_score/100
average_ap_cluster = mean_ap_num_cluster/100
average_ap_score = mean_ap_score/100
average_km_score = mean_km_score/100
print("average Agglomerative cluster",average_acl_cluster)
print(f'FMI_Agglomerative= {average_acl_score:.3f}')
print("average Affinity Propagation cluster",average_ap_cluster)
print(f'FMI_Affinity_Propagation = {average_ap_score:.3f}')
print(f'FMI_k_medoids = {average_km_score:.3f}')

C:\Users\hejx\AppData\Local\Temp\ipykernel_1148\643376983.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate_1, data_1 = wavfile.read('wav//220747__jmuehlhans__tuning-fork-440-hz-resonance-box.wav')
C:\Users\hejx\AppData\Local\Temp\ipykernel_1148\643376983.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate_2, data_2 = wavfile.read('wav//334536__teddy_frost__piano-normal-d4.wav')
Spectrum:   1%|▋                                                                     | 1/100 [01:23<2:18:21, 83.85s/it]

acl_score 1.0
len(set(ap_labels)) 2
ap_score 0.9318544609714189
km_score 1.0


Spectrum:   2%|█▍                                                                    | 2/100 [02:51<2:20:09, 85.81s/it]

acl_score 1.0
len(set(ap_labels)) 2
ap_score 0.799063251002448
km_score 1.0


Spectrum:   3%|██                                                                    | 3/100 [04:17<2:19:11, 86.10s/it]

...

acl_score 1.0
len(set(ap_labels)) 2
ap_score 0.821215566341779
km_score 1.0


Spectrum:  99%|████████████████████████████████████████████████████████████████████▎| 99/100 [2:15:43<01:23, 83.01s/it]

acl_score 1.0
len(set(ap_labels)) 2
ap_score 0.9412598950343314
km_score 1.0


Spectrum: 100%|████████████████████████████████████████████████████████████████████| 100/100 [2:17:02<00:00, 82.22s/it]

acl_score 1.0
len(set(ap_labels)) 2
ap_score 0.8610654408198618
km_score 1.0
average Agglomerative cluster 2.0
FMI_Agglomerative= 1.000
average Affinity Propagation cluster 2.0
FMI_Affinity_Propagation = 0.874
FMI_k_medoids = 1.000
